# Test Tiny A11y Model (LoRA)
This notebook loads the base DeepSeek-Coder model, applies your LoRA fine-tuned weights (`younglim/tiny-a11y-model`), and lets you test prompts interactively.

Requirements: Free GPU runtime in Colab (T4 recommended).

In [ ]:
# Install required packages
!pip install --upgrade pip
!pip install transformers peft datasets accelerate huggingface_hub

In [ ]:
# Hugging Face login
from huggingface_hub import notebook_login
notebook_login()  # Paste your HF token when prompted

In [ ]:
# Load base model and LoRA fine-tuned weights
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Base model
base_model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Apply LoRA adapter
lora_model_name = 'younglim/tiny-a11y-model'
model = PeftModel.from_pretrained(base_model, lora_model_name)

# Ensure pad_token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# Helper function to run the model
def ask_model(prompt, max_tokens=150):
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Example prompt
prompt = 'Explain why alt text is important for images.'
answer = ask_model(prompt)
print('Prompt:', prompt)
print('Answer:', answer)

✅ You can now replace the prompt above with any accessibility-related question or text you want the Tiny A11y model to answer.